In [1]:
import tensorflow.compat.v1 as tf
import numpy as np
import utils.read_mnist as read_mnist
import time
import lenet

把数据读取出来并返回
这里提供两个函数
分别是read_mnist和read_fashion_mnist

In [2]:
print(read_mnist.read_mnist.__doc__)

返回mnist数据，两个参数分别是one_hot和standard来控制是否将数据归一化和变成OneHot
        分别返回训练数据，训练标签，测试数据，测试标签


In [3]:
print(read_mnist.read_fashion_mnist.__doc__)

返回fashion_mnist数据，两个参数分别是one_hot和standard来控制是否将数据归一化和变成OneHot
        分别返回训练数据，训练标签，测试数据，测试标签，按照标签获得名字的一个字典


In [4]:
train_x, train_y, test_x, test_y = read_mnist.read_mnist(one_hot=True, standard=True)
index_train = np.random.permutation(train_x.shape[0])
train_x, train_y = train_x[index_train], train_y[index_train]

In [5]:
print('训练数据有{}个,维度为{}行，{}列'.format(train_x.shape[0],train_x.shape[1],train_x.shape[2]))
print('测试数据有{}个,维度为{}行，{}列'.format(test_x.shape[0],test_x.shape[1],test_x.shape[2]))

训练数据有60000个,维度为28行，28列
测试数据有10000个,维度为28行，28列


In [7]:
gpu_options = tf.GPUOptions(allow_growth=True)     #允许动态申请显存
tf.GPUOptions(per_process_gpu_memory_fraction=0.4) # 限制GPU显存使用率是40%

per_process_gpu_memory_fraction: 0.4

In [8]:
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
model = lenet.LeNet(lr_rate=0.001, regular=0.0001)
sess.run(tf.global_variables_initializer())

In [9]:
epochs = 20
batch_size = 64
saver = tf.train.Saver()
lr_rate = 0.001
for epoch in range(epochs):
    start = time.time()
    batch_sofar = 0
    for j in range(train_x.shape[0] // batch_size + 1):
        model.train(train_x=train_x[batch_sofar:batch_sofar + batch_size, :, :, :],
                    train_y=train_y[batch_sofar:batch_sofar + batch_size, :],
                    session=sess)
        batch_sofar += batch_size
        if batch_sofar > train_x.shape[0]:
            model.train(train_x=train_x[batch_sofar - batch_size:, :, :, :],
                        train_y=train_y[batch_sofar - batch_size:, :],
                        session=sess)
    if (epoch + 1) / 50 == 0:
        lr_rate = lr_rate / 2
        model.change_lr_rate(lr_rate)
    #rs = sess.run(merge, feed_dict={model.input_x: test_x, model.input_y: test_y})
    #file_writer.add_summary(rs, epoch)
    print('{:.0f} epoch use {:.2f} ms'.format(epoch, (time.time()-start)*1000))
    print('{:.0f} epoch loss on test data is {:.2f} '.format(epoch, model.compute_loss(test_x, test_y, session=sess)))
    print('{:.0f} epoch accuary on test data is {:.2f}% '.format(epoch, model.compute_accuracy(test_x, test_y, session=sess)*100))
saver.save(sess, r'ckpt/lenet-ckpt', global_step=epoch)
sess.close()

0 epoch use 4556.00 ms
0 epoch loss on test data is 2.25 
0 epoch accuary on test data is 24.81% 
1 epoch use 2824.00 ms
1 epoch loss on test data is 1.79 
1 epoch accuary on test data is 60.10% 
2 epoch use 2917.00 ms
2 epoch loss on test data is 0.69 
2 epoch accuary on test data is 82.66% 
3 epoch use 2934.97 ms
3 epoch loss on test data is 0.45 
3 epoch accuary on test data is 87.69% 
4 epoch use 2922.96 ms
4 epoch loss on test data is 0.38 
4 epoch accuary on test data is 89.45% 
5 epoch use 2924.96 ms
5 epoch loss on test data is 0.34 
5 epoch accuary on test data is 90.42% 
6 epoch use 2932.00 ms
6 epoch loss on test data is 0.31 
6 epoch accuary on test data is 91.18% 
7 epoch use 2938.96 ms
7 epoch loss on test data is 0.29 
7 epoch accuary on test data is 91.87% 
8 epoch use 2924.00 ms
8 epoch loss on test data is 0.27 
8 epoch accuary on test data is 92.34% 
9 epoch use 2921.00 ms
9 epoch loss on test data is 0.25 
9 epoch accuary on test data is 92.88% 
10 epoch use 2944.98